In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" #6,7import pandas as pd 2080s = 0,3,5,6,8 Nvidia-smi ids: 0, 3, 5, 6, 8 Actual id: 5,6,7,8,9 

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
    PromptTuningConfig)

import evaluate
from seqeval.metrics import f1_score, precision_score, recall_score
from datasets import load_dataset, load_from_disk, load_metric
from transformers import (AutoModel,AutoModelForSequenceClassification,
                          AutoModelForTokenClassification, AutoTokenizer,
                          DataCollatorForTokenClassification, Trainer, TrainingArguments,
                          get_linear_schedule_with_warmup, set_seed)
from tqdm import tqdm
from loguru import logger as loguru_logger
import numpy as np

import sys
sys.path.append("../")

/home/upamanyu/.pyenv/versions/3.9.15/envs/eff_ml/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
model_name = 'zhihan1996/DNABERT-2-117M'
peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, 
                        inference_mode=False, r=8, lora_alpha=16, 
                        lora_dropout=0.1, bias="all", 
                        target_modules=['Wqkv'])

In [32]:
# model = AutoModelForSequenceClassification.from_pretrained(model_name, 
#                                                     trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
model = get_peft_model(model, peft_config=peft_config)
# model.print_trainable_parameters()

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Some weights of the model checkpoint at zhihan1996/DNABERT-2-117M were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to b

In [13]:
c = list(model.named_children())

In [30]:
list(c[1][1].named_children())[0][1][11]

BertLayer(
  (attention): BertUnpadAttention(
    (self): BertUnpadSelfAttention(
      (dropout): Dropout(p=0.0, inplace=False)
      (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (mlp): BertGatedLinearUnitMLP(
    (gated_layers): Linear(in_features=768, out_features=6144, bias=False)
    (act): GELU(approximate='none')
    (wo): Linear(in_features=3072, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (layernorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  )
)

In [4]:
dataset = load_dataset('csv', name='prom_core_tata',
                       data_files={'train': '/mnt/sdd/efficient_ml_data/datasets/GUE/prom/prom_core_tata/train.csv',
                                   'test': '/mnt/sdd/efficient_ml_data/datasets/GUE/prom/prom_core_tata/test.csv'})
collate_fn = DataCollatorForTokenClassification()

Found cached dataset csv (/home/upamanyu/.cache/huggingface/datasets/csv/prom_core_tata-c63ad107f9ad8687/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 2/2 [00:00<00:00, 503.40it/s]
